<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3175cfcd3998315e3b43c2c813092a6b91b994988425b31d2bd9098f3abf5fe7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-09 14:45:34
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -2.72 K (-0.02%)
Current PnL: -29.35 L (-18.96%)
CY Booked + Current PnL: -15.06 L (-9.73%)
-------------------
Total profit:  1.14 L
Total loss:  -30.50 L
-------------------
Total Booked + Current PnL: 11.92 L (9.32%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.29%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.99 L (67.65%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.39,-17.97,22.02,0.09,18203.0,-18109.0,82668.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.61,OX40N,NTT,DURABLES
43,ITC,452.00,-0.16,-1.86,12.54,10.44,24629.0,-3731.0,196407.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.47,3.60,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,0.34,-14.19,16.57,0.02,24819.0,-24779.0,149780.0,128.24,47.0,M-SC,16.66,234.0,-1.00,1.12,30.28,OX40N,NTT,PAINTS
50,MASFIN,398.61,1.40,-3.43,26.38,22.05,24961.0,-3360.0,94620.0,-16.89,53.0,H-SC,7.01,164.0,-0.13,0.71,37.54,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.67,4.62,14.88,20.19,27138.0,8062.0,182382.0,-17.91,43.0,X-MC,6.16,68.0,0.30,1.36,20.44,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELAXO,1176.00,4.12,-46.35,183.89,52.31,143212.0,-67281.0,77879.0,-42.89,48.0,X-SC,5.97,91.0,-0.47,0.58,4.25,X40N,NTT,FOOTWEAR
10,BANDHANBNK,400.00,3.65,-32.45,184.86,92.42,347838.0,-90396.0,188163.0,32.18,33.0,H-SC,7.87,170.0,-0.26,1.40,8.74,XY24,NTT,BANKS
56,RAJESHEXPO,518.00,3.65,-63.73,175.93,0.07,87522.0,-87429.0,49748.0,1723.23,52.0,L-SC,2.36,268.0,-1.00,0.37,23.73,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,3.49,-43.93,210.60,74.15,193188.0,-71873.0,91732.0,-265.32,46.0,H-SC,21.18,149.0,-0.37,0.68,17.59,XR,NTT,CHEMICALS
71,TANLA,1943.92,2.55,-38.12,255.05,119.69,427298.0,-103226.0,167535.0,-37.58,34.0,H-SC,7.77,156.0,-0.24,1.25,32.35,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.26,-4.31,-2.06,23.48,20.95,57900.0,-5174.0,246594.0,-3.02,49.0,X-LC,19.03,31.0,-0.09,1.84,32.85,X40,ATH,PAINTS
45,JIOFIN,387.00,-2.16,-4.40,30.22,24.48,77240.0,-11773.0,255592.0,-6.04,40.0,X-LC,6.44,36.0,-0.15,1.91,48.15,XY24,BTT,FINANCE
2,ABBOTINDIA,35195.00,-2.12,-6.19,25.18,17.44,35397.0,-9272.0,140575.0,-19.22,32.0,X-MC,4.57,63.0,-0.26,1.05,11.78,X40,ATH,PHARMA
49,LAOPALA,464.00,-2.11,-39.38,129.44,39.09,102874.0,-51629.0,79476.0,73.50,24.0,H-SC,7.38,155.0,-0.50,0.59,2.36,AR,NTT,CERAMICS
28,GREENPANEL,537.00,-2.01,-39.23,135.55,43.14,150805.0,-71824.0,111254.0,149.35,22.0,M-SC,8.65,240.0,-0.48,0.83,4.65,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.40,-7.89,125.72,107.91,170974.0,-11648.0,135996.0,-25.01,32.0,M-SC,11.86,216.0,-0.07,1.02,1.40,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.34,-14.19,16.57,0.02,24819.0,-24779.0,149780.0,128.24,47.0,M-SC,16.66,234.0,-1.00,1.12,30.28,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.39,-17.97,22.02,0.09,18203.0,-18109.0,82668.0,91.89,32.0,M-SC,2.19,253.0,-0.99,0.62,8.61,OX40N,NTT,DURABLES
66,SIS,528.00,0.71,-24.65,60.93,21.26,50978.0,-27366.0,83666.0,1973.06,47.0,H-SC,4.59,166.0,-0.54,0.62,13.04,OX40N,NTT,MISC
47,KANSAINER,340.00,0.49,-25.59,52.51,13.47,105359.0,-69021.0,200646.0,-69.48,28.0,H-SC,3.20,158.0,-0.66,1.50,2.02,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.48,5.61,63.11,72.27,112943.0,9514.0,178962.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.14,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.48,5.61,63.11,72.27,112943.0,9514.0,178962.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.14,XR,NTT,IT
38,INDIAMART,4810.62,-0.76,-3.05,113.22,106.72,135387.0,-3757.0,119579.0,-51.96,33.0,H-SC,1.19,139.0,-0.03,0.89,19.62,AR,ATH,MISC
25,FINCABLES,1641.55,1.40,-7.89,125.72,107.91,170974.0,-11648.0,135996.0,-25.01,32.0,M-SC,11.86,216.0,-0.07,1.02,1.40,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.81,-3.29,40.24,35.63,81558.0,-6895.0,202679.0,-16.74,34.0,H-MC,3.17,119.0,-0.08,1.51,12.83,AR,ATH,PHARMA
37,IEX,219.00,-0.99,-4.64,55.86,48.63,106509.0,-9282.0,190672.0,-36.48,46.0,H-SC,14.09,136.0,-0.09,1.42,6.37,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,1.89,-21.95,90.21,48.45,141169.0,-44017.0,156489.0,-27.60,21.0,X-MC,11.18,64.0,-0.31,1.17,1.89,X40N,ATH,FINANCE
22,DIXON,18931.72,-0.66,-13.47,40.52,21.58,49133.0,-18882.0,121257.0,-56.34,24.0,X-MC,5.47,56.0,-0.38,0.91,7.35,X40N,ATH,IT
75,TMPV,600.00,-0.95,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.53,-10.94,29.37,15.23,52407.0,-21911.0,178436.0,-23.85,29.0,X-MC,7.89,66.0,-0.42,1.33,17.79,X40N,NTT,REALTY
33,HINDUNILVR,2922.00,-0.25,-10.21,26.59,13.67,61375.0,-26243.0,230820.0,-16.94,29.0,X-LC,2.98,14.0,-0.43,1.72,8.97,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-1.68,-22.02,96.83,53.50,227853.0,-66439.0,235312.0,-62.23,31.0,X-MC,1.24,58.0,-0.29,1.76,4.52,XY24,NTT,FMCG
43,ITC,452.0,-0.16,-1.86,12.54,10.44,24629.0,-3731.0,196407.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.47,3.60,X40,NTT,FMCG
3,ACC,3906.0,-0.40,-24.82,118.35,64.15,211716.0,-59061.0,178890.0,-55.88,36.0,X-SC,1.78,82.0,-0.28,1.34,0.35,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.20,-80.49,539.71,24.79,265160.0,-202726.0,49130.0,-81.32,38.0,X-SC,1.81,86.0,-0.76,0.37,20.11,X40N,NTT,MISC
36,ICICIPRULI,790.0,1.14,3.33,26.74,30.97,48834.0,5889.0,182627.0,-21.03,55.0,X-MC,2.06,75.0,0.12,1.36,16.31,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.07,-0.17,-9.20,38.84,26.07,57747.0,-15060.0,148680.0,-33.14,30.0,X-MC,11.48,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.95,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
54,PGHH,17907.65,-1.00,-5.89,42.14,33.76,79638.0,-11835.0,188985.0,-33.58,34.0,X-MC,5.09,57.0,-0.15,1.41,0.10,X40,ATH,FMCG
3,ACC,3906.00,-0.40,-24.82,118.35,64.15,211716.0,-59061.0,178890.0,-55.88,36.0,X-SC,1.78,82.0,-0.28,1.34,0.35,XY24,BTT,CEMENT
31,HAVELLS,2069.16,0.66,-9.30,46.07,32.48,137052.0,-30505.0,297486.0,-13.91,40.0,X-MC,5.19,67.0,-0.22,2.22,1.06,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.78,-29.81,103.50,42.84,47226.0,-19377.0,45629.0,-54.27,41.0,X-SC,37.01,83.0,-0.41,0.34,3.23,XY24,NTT,MISC
15,CAMS,4762.00,-1.20,-80.49,539.71,24.79,265160.0,-202726.0,49130.0,-81.32,38.0,X-SC,1.81,86.0,-0.76,0.37,20.11,X40N,NTT,MISC
59,RELAXO,1176.00,4.12,-46.35,183.89,52.31,143212.0,-67281.0,77879.0,-42.89,48.0,X-SC,5.97,91.0,-0.47,0.58,4.25,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,1.32,-38.01,118.11,35.20,94208.0,-48907.0,79763.0,4.55,30.0,X-SC,15.27,92.0,-0.52,0.60,1.32,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.66,-13.47,40.52,21.58,49133.0,-18882.0,121257.0,-56.34,24.0,X-MC,5.47,56.0,-0.38,0.91,7.35,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.87,-12.82,36.82,19.28,111049.0,-44357.0,301599.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.50,X40,ATH,IT
41,INFY,2275.00,-0.90,8.67,42.52,54.87,144573.0,27124.0,340012.0,-15.29,63.0,X-LC,3.38,2.0,0.19,2.54,17.67,X40,BTT,IT
75,TMPV,600.00,-0.95,-18.25,73.91,42.18,167018.0,-50435.0,225975.0,-26.74,25.0,X-LC,5.10,3.0,-0.30,1.69,0.00,XY24,NTT,AUTO
81,VBL,671.64,-0.33,-5.41,43.44,35.67,129774.0,-17098.0,298744.0,-16.71,50.0,X-LC,4.83,4.0,-0.13,2.23,7.69,X40N,ATH,FMCG
43,ITC,452.00,-0.16,-1.86,12.54,10.44,24629.0,-3731.0,196407.0,-41.56,43.0,X-LC,1.66,5.0,-0.15,1.47,3.60,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.03,-19.43,125.48,81.67,94844.0,-18225.0,75585.0,6788.89,32.0,L-SC,14.88,271.0,-0.19,0.56,46.37,XR,NTT,CERAMICS
50,MASFIN,398.61,1.40,-3.43,26.38,22.05,24961.0,-3360.0,94620.0,-16.89,53.0,H-SC,7.01,164.0,-0.13,0.71,37.54,XR,ATH,FINANCE
13,BSOFT,831.70,0.63,-21.89,93.04,50.79,102220.0,-30786.0,109867.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.92,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.34,-14.19,16.57,0.02,24819.0,-24779.0,149780.0,128.24,47.0,M-SC,16.66,234.0,-1.00,1.12,30.28,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.26,-4.31,-2.06,23.48,20.95,57900.0,-5174.0,246594.0,-3.02,49.0,X-LC,19.03,31.0,-0.09,1.84,32.85,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.63,-21.89,93.04,50.79,102220.0,-30786.0,109867.0,0.12,70.0,H-SC,5.84,151.0,-0.30,0.82,27.92,XR,ATH,IT
32,HCLTECH,1922.01,-1.72,7.70,15.81,24.73,41194.0,18633.0,260557.0,12.34,62.0,X-LC,5.75,7.0,0.45,1.94,24.11,X40,ATH,IT
84,WIPRO,420.00,-1.48,5.61,63.11,72.27,112943.0,9514.0,178962.0,-9.26,64.0,M-LC,2.23,99.0,0.08,1.34,12.14,XR,NTT,IT
73,TCS,4389.97,-0.87,-12.82,36.82,19.28,111049.0,-44357.0,301599.0,-23.95,63.0,X-LC,5.77,1.0,-0.40,2.25,11.50,X40,ATH,IT
5,ASIANPAINT,3460.26,-4.31,-2.06,23.48,20.95,57900.0,-5174.0,246594.0,-3.02,49.0,X-LC,19.03,31.0,-0.09,1.84,32.85,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.80
1,25,45.75
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.98
MC    30.32
LC    24.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.31
X40      22.51
X40N     10.90
XR       10.05
XY25      9.40
AR        9.32
OX40N     7.77
SR        1.02
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.50
X-MC    23.61
X-LC    21.16
M-SC    12.09
X-SC     7.01
H-MC     4.95
M-MC     1.40
L-SC     1.38
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.42,-7.86,42.75
IT,12.85,-17.14,80.47
FINANCE,9.54,-21.34,72.61
ELECTRICAL,6.24,-12.28,53.41
MISC,6.09,-58.91,124.01
PAINTS,6.09,-13.23,30.06
INSURANCE,4.63,-1.79,37.66
PHARMA,4.09,-1.94,34.57
AUTO,3.41,-25.98,80.00


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3245562.0,21
AR,1323199.0,10
XR,1315434.0,13
X40,1022653.0,14
X40N,989220.0,9
OX40N,752180.0,10
XY25,382159.0,6
SR,284111.0,2
MH,84049.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3695142.0,25
M-SC,1441620.0,15
X-MC,1321458.0,16
X-SC,982705.0,8
X-LC,927880.0,11
H-MC,414322.0,3
L-SC,275888.0,3
M-LC,112943.0,1
H-LC,75819.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295455.0      6
           AR         900967.0      5
M-SC       XY24       819811.0      6
H-SC       XR         783455.0      7
X-SC       X40N       505221.0      3
X-MC       X40        458302.0      7
           XY24       397519.0      3
X-LC       X40        379345.0      5
H-SC       OX40N      347105.0      4
M-SC       OX40N      317553.0      5
X-SC       XY24       292478.0      3
H-SC       SR         284111.0      2
X-MC       X40N       279248.0      4
X-LC       XY24       244258.0      2
H-MC       AR         218281.0      2
X-LC       X40N       204751.0      2
H-MC       XY24       196041.0      1
L-SC       XR         188366.0      2
X-MC       XY25       186389.0      2
X-SC       X40        185006.0      2
M-SC       XR         176124.0      2
           AR         128132.0      2
M-LC       XR         112943.0      1
X-LC       XY25        99526.0      2
L-SC       OX40N       87522.0      1
H-SC       MH          84049.0      1
H-LC       AR          75819.0      1
M-MC       XY25        57201.0      1
L-MC       XR          54546.0      1
L-LC       XY25        39043.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
